In [1]:
import tensorflow as tf
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dropout, Rescaling
from tensorflow.keras.models import Sequential #Feed-Forward-NN
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt #Plot Images
import os #Access files and folders
import numpy as np #Calculations
from datetime import datetime

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("GPUs detected: ", len(physical_devices))
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True) #Use GPU if available

In [ ]:
path = "G:/Python/data/10_animals" #Path to dataset
height = 300; width = 300 #Rescale images in dataset to set size
batch_size = 64 #determine batch size for training

train = tf.keras.preprocessing.image_dataset_from_directory( #Loading training-data
    path,
    image_size=(height, width),
    validation_split=0.25, #Determine how many images will be used for validation
    subset="training",
    seed=123, #Shuffling and Transformation
    batch_size=batch_size
)

val = tf.keras.preprocessing.image_dataset_from_directory( #Loading validation-data
    path,
    image_size=(height, width),
    validation_split=0.25,
    subset="validation",
    seed=123,
    batch_size=batch_size
)

classes = train.class_names

In [ ]:
print(classes)

In [2]:
#Data-Augmentation
data_aug = Sequential([ 
    tf.keras.layers.experimental
    .preprocessing.Rescaling(1./255),

    tf.keras.layers.experimental
    .preprocessing.RandomFlip("horizontal_and_vertical"),

    tf.keras.layers.experimental
    .preprocessing.RandomRotation(0.3),

    tf.keras.layers.experimental.preprocessing
    .RandomZoom(height_factor=(-0.05, -0.15), width_factor=(-0.05, -0.15))
])

In [ ]:
#Plot random images from dataset
data_iterator = train.as_numpy_iterator() 
batch = data_iterator.next()

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(classes[batch[1][idx]])

In [ ]:
model = Sequential([
    InputLayer((height, width, 3)), #Input-Layer
    data_aug, #Data Augmentation

    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),

    Conv2D(64, (3,3), padding='same', activation='relu'), #Convolution-Layer
    MaxPooling2D(), #Pooling-Layer
    Dropout(0.25), #Dropout-Layer

    Conv2D(128, (3,3), padding='same', activation='relu'), #Convolution-Layer
    MaxPooling2D(), #Pooling-Layer
    Dropout(0.25), #Dropout-Layer

    Conv2D(128, (3,3), padding='same', activation='relu'), #Convolution-Layer
    MaxPooling2D(), #Pooling-Layer
    Dropout(0.25), #Dropout-Layer

    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(classes), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'] #tracking for accuracy
)

model.summary() #overview of built model

In [11]:
model_path = "C:/Users/maxis/Desktop/Proseminar - CNN/checkpoints/Checkpoint-{epoch:02d}--{val_accuracy:.02f}.h5"
checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', varbose=1, save_best_only=True, mode='max')

In [12]:
model_state = f"animals_10_{datetime.now().strftime('%H-%M-%S')}"
tensorboard = TensorBoard(log_dir = f"C:/Users/maxis/Desktop/Proseminar - CNN/logdir/{model_state}")

In [ ]:
model.fit(train, validation_data=val, epochs=10, callbacks=[tensorboard, checkpoint]) #Train the model

In [ ]:
img = tf.keras.preprocessing.image.load_img("G:/Python/data/img_from_google/cat1.jpg", target_size=(height, width)) #Load image thats not already in the dataset

plt.imshow(img) #Plot image
plt.show()

img = tf.keras.preprocessing.image.img_to_array(img) #Convert image into array
img = np.expand_dims(img, axis=0) #Add dimensions to array to be processed by the model

prediction = model.predict(img) #Model predicts class of image

print()

prediction_probabilities = tf.math.top_k(prediction, k=5) #Get top-5 predictions
top_5_scores = prediction_probabilities.values.numpy().tolist() #Get top-5 probabilities
dict_class_entries = prediction_probabilities.indices.numpy().tolist() #Get matching classes to probabilities
for label, confidence in zip(dict_class_entries[0], top_5_scores[0]):
    print(f"Image predicted to be {classes[label]} with confidence of {round(confidence*100, 3)}%") #Printing the class and probability

In [ ]:
model.save(os.path.join("C:/Users/maxis/Desktop/Proseminar - CNN/Models","animals_10_69_val_accuracy.h5")) #Save the structure (weights etc.) of the model